<a href="https://colab.research.google.com/github/VicDc/VIC_/blob/main/VDC_A3_Smart_Home_AI_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -----------------------------------------------------------------------------
# Install dependencies manually if not already installed:
# pip install transformers torch
# -----------------------------------------------------------------------------
import torch
from transformers import AutoTokenizer, AutoModel
import re

print("Attempting to load Hugging Face model...")
# -----------------------------------------------------------------------------
# AI Integration with Hugging Face
# -----------------------------------------------------------------------------
try:
    model_name = "Musixmatch/umberto-wikipedia-uncased-v1"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    print(f"Hugging Face model loaded successfully: {model_name}")
except Exception as e:
    print(f"Error loading Hugging Face model: {e}")
    print("Please ensure 'transformers' and 'torch' are installed and you have an internet connection.")
    # Exit if model loading fails, as it's a core part mentioned
    exit()

def embed_command(command: str):
    """Generate a semantic embedding for the input command."""
    inputs = tokenizer(command, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad(): # It's good practice for inference
        outputs = model(**inputs)
    # Mean pooling over token embeddings
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

class Light:
    VALID_STATES = ('on', 'off')

    def __init__(self):
        self.state = 'off'

    def set_state(self, state):
        if state in self.VALID_STATES:
            self.state = state
            print(f"Light turned {state}.")
        else:
            print(f"Invalid state for Light: {state}")

    def get_status(self):
        print(f"Light is currently {self.state}.")


class Fan:
    VALID_SPEEDS = ('low', 'medium', 'high')

    def __init__(self):
        self.speed = 'off' # Initial state is off

    def set_state(self, state):
        if state in ('on', 'off'):
            if state == 'off':
                self.speed = 'off'
                print("Fan turned off.")
            else: # 'on'
                self.speed = 'medium' # Default to medium when turned on
                print("Fan turned on at medium speed.")
        elif state in self.VALID_SPEEDS:
            self.speed = state
            print(f"Fan speed set to {state}.")
        else:
            print(f"Invalid state or speed for Fan: {state}")

    def get_status(self):
        print(f"Fan is set to {self.speed}.")


class Thermostat:
    def __init__(self):
        self.temperature = 20.0

    def set_temperature(self, temp_str):
        try:
            temp = float(temp_str)
            self.temperature = temp
            print(f"Thermostat set to {temp}°C.")
        except ValueError:
            print(f"Invalid temperature value: {temp_str}")

    def get_status(self):
        print(f"Thermostat is currently set at {self.temperature}°C.")

def parse_command(command):
    command_lower = command.lower()

    # Esempio di come potresti integrare l'embedding in futuro (attualmente non usato):
    # if "qualcosa di complesso" in command_lower:
    # embedding = embed_command(command)
    # Qui potresti usare l'embedding per una comprensione più sfumata del comando
    # print(f"Embedding for '{command}': {embedding}") # Solo per debug

    if "light" in command_lower or "luce" in command_lower:
        device = 'light'
        if "status" in command_lower or "stato" in command_lower:
            return {'intent': 'get_status', 'device_type': device}
        if "turn on" in command_lower or "switch on" in command_lower or "accendi" in command_lower:
            return {'intent': 'set_state', 'device_type': device, 'value': 'on'}
        if "turn off" in command_lower or "switch off" in command_lower or "spegni" in command_lower:
            return {'intent': 'set_state', 'device_type': device, 'value': 'off'}

    if "fan" in command_lower or "ventilatore" in command_lower:
        device = 'fan'
        if "status" in command_lower or "stato" in command_lower:
            return {'intent': 'get_status', 'device_type': device}
        if "turn on" in command_lower or "switch on" in command_lower or "accendi" in command_lower:
            # Quando si accende il ventilatore genericamente, si imposta una velocità predefinita (es. 'on' che la classe Fan gestisce)
            return {'intent': 'set_state', 'device_type': device, 'value': 'on'}
        if "turn off" in command_lower or "switch off" in command_lower or "spegni" in command_lower:
            return {'intent': 'set_state', 'device_type': device, 'value': 'off'}
        for speed in Fan.VALID_SPEEDS: # "low", "medium", "high"
            if speed in command_lower or \
               (speed == 'low' and "bassa" in command_lower) or \
               (speed == 'medium' and "media" in command_lower) or \
               (speed == 'high' and "alta" in command_lower):
                return {'intent': 'set_state', 'device_type': device, 'value': speed}

    if "thermostat" in command_lower or "temperature" in command_lower or "termostato" in command_lower or "temperatura" in command_lower:
        device = 'thermostat'
        if "status" in command_lower or "stato" in command_lower:
            return {'intent': 'get_status', 'device_type': device}
        # Cerca un numero nel comando, che potrebbe essere la temperatura
        match = re.search(r'(\d+\.?\d*|\d*\.?\d+)', command_lower) # Supporta decimali
        if match:
            # Verifica che ci sia un'intenzione di impostare la temperatura (es. "set", "imposta")
            if "set" in command_lower or "imposta" in command_lower or "metti a" in command_lower or "porta a" in command_lower or \
               any(char.isdigit() for char in command_lower.split()[-1]): # se l'ultimo "token" è un numero
                return {'intent': 'set_temperature', 'device_type': device, 'value': match.group(1)}


    if "exit" in command_lower or "quit" in command_lower or "esci" in command_lower:
        return {'intent': 'exit'}

    # Se non riconosce il dispositivo ma c'è un comando di stato, potrebbe essere generico
    if "status" in command_lower or "stato" in command_lower:
        # Potresti chiedere all'utente di specificare per quale dispositivo
        print("Per quale dispositivo vuoi conoscere lo stato? (light, fan, thermostat)")
        return {'intent': 'clarify_status'}


    return {'intent': 'unrecognized'}

def execute(parsed, devices):
    intent = parsed.get('intent')
    device_type = parsed.get('device_type')

    if intent == 'exit':
        print("Exiting Smart Home CLI.")
        return False # Segnala di terminare il loop

    if intent == 'unrecognized':
        print("Sorry, I didn't understand that command.")
        # Esempio di utilizzo dell'embedding per comandi non riconosciuti
        # command_embedding = embed_command(last_user_command) # necessita di passare l'ultimo comando
        # print(f"Embedding of unrecognized command: {command_embedding}")
        # Qui potresti cercare comandi simili o suggerire alternative
        return True

    if intent == 'clarify_status':
        # Già gestito in parse_command con una print, qui non fa nulla se non attendere il prossimo comando
        return True

    if not device_type and intent not in ['exit', 'unrecognized', 'clarify_status']:
        print("Device not specified or recognized in the command.")
        return True

    device = devices.get(device_type)
    if not device:
        print(f"Device type '{device_type}' not found in the system.")
        return True

    if intent == 'get_status':
        device.get_status()
    elif intent == 'set_state':
        device.set_state(parsed.get('value'))
    elif intent == 'set_temperature' and device_type == 'thermostat':
        # Assicura che set_temperature sia chiamato solo per il termostato
        device.set_temperature(parsed.get('value'))
    else:
        # Questo caso potrebbe non essere mai raggiunto se parse_command è robusto
        print(f"Action '{intent}' not clearly applicable or understood for device '{device_type}'.")

    return True # Segnala di continuare il loop

def run_smart_home_cli():
    """
    Starts and manages the Smart Home CLI.
    """
    devices = {
        'light': Light(),
        'fan': Fan(),
        'thermostat': Thermostat()
    }
    print("\nWelcome to the Smart Home CLI. Type 'exit' or 'esci' to quit.")
    print("Example commands:")
    print("  luce on / luce off / stato luce")
    print("  ventilatore on / ventilatore off / ventilatore low / stato ventilatore")
    print("  termostato 22 / termostato imposta 23.5 / stato termostato")
    print("  embed il mio comando personalizzato <--- per testare la funzione di embedding\n")


    while True:
        command = input(">> ")
        if not command.strip(): # Ignora input vuoti
            continue

        # Comando speciale per testare la funzione di embedding direttamente
        if command.lower().startswith("embed "):
            try:
                text_to_embed = command[len("embed "):]
                if text_to_embed:
                    embedding_vector = embed_command(text_to_embed)
                    print(f"Embedding vector for '{text_to_embed}':")
                    print(embedding_vector)
                    # Potresti voler stampare solo una parte se è molto lungo
                    # print(embedding_vector[:, :5]) # Esempio: prime 5 dimensioni
                else:
                    print("Please provide text after 'embed ' to get its embedding.")
            except Exception as e:
                print(f"Error during embedding: {e}")
            continue


        parsed = parse_command(command)
        if not execute(parsed, devices): # execute ritorna False solo per 'exit'
            break

if __name__ == '__main__':
    # Questo blocco viene eseguito solo quando lo script è lanciato direttamente
    run_smart_home_cli()

Attempting to load Hugging Face model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/801k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Hugging Face model loaded successfully: Musixmatch/umberto-wikipedia-uncased-v1

Welcome to the Smart Home CLI. Type 'exit' or 'esci' to quit.
Example commands:
  luce on / luce off / stato luce
  ventilatore on / ventilatore off / ventilatore low / stato ventilatore
  termostato 22 / termostato imposta 23.5 / stato termostato
  embed il mio comando personalizzato <--- per testare la funzione di embedding



model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Light is currently off.
Thermostat is currently set at 20.0°C.
Thermostat set to 26.0°C.
Light turned on.
